<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 10.3 PySpark for Big Data

## Introduction

**Note**: this notebook is to be run in Google Colab on your Google Drive. It will not work locally on your computer.

The purpose of this lab is to gain further exposure to cloud computing, often necessary when datasets become too large to manage on a local machine. You will learn how to work with a large dataset through the PySpark Python library with Google Colaboratory (Colab).

In Google Colab, a virtual machine is automatically set up to execute your code. The maximum lifetime of such a machine is 12 hours. Note that notebooks will be disconnected from virtual machines if left idle. If this happens simple click on the Connect button to reconnect. If the kernel needs to be restarted (via the Runtime menu), variables may be lost but packages would not need to be reinstalled unless a new machine is assigned.

https://research.google.com/colaboratory/faq.html

Sign into colab.research.google.com and choose the Upload tab and upload this notebook.  This will automatically create a folder called "Colab Notebooks" in your Google Drive (if it does not already exist).

Apache Spark is an open-source cluster-computing framework, able to work with large datasets quickly by performing in-memory caching and computation. Pyspark is a Python API for Spark commonly used to manipulate big data. For reference one useful cheat sheet is available at https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf


Fortunately Pyspark is straightforward to setup in Google Colab:

In [ ]:
!pip install pyspark py4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=d3135975c17c34a1216b2942b9317b8bd49306ea0cd452261b3d50fecda5fe1b
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


To work with Spark DataFrames we firstly need to create a Spark DataFrame:

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, trim, avg, round, when
from pyspark import SparkFiles

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
spark = SparkSession.builder.appName("populationdata").getOrCreate()

In [ ]:
spark

## Loading the dataset and EDA

The dataset to be analysed in this lab shows population estimates by age and gender:

   - PopMale: Male population for the individual age (thousands)
   - PopFemale: Female population for the individual age (thousands)
   - PopTotal: Total population for the individual age (thousands)

Further details can be found at https://population.un.org/wpp/.

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# run this cell after uploading the file into your Colab Notebooks folder
df = spark.read.csv(r"/content/drive/MyDrive/Colab Notebooks/WPP2019_PopulationBySingleAgeSex_1950-2019.csv", header=True)

In [ ]:
type(df)

In [ ]:
df

Note that unlike Pandas this does not display a preview of the dataset, only the schema. This is because Spark performs lazy evaluation, only displaying rows when needed.

In [ ]:
df.columns

**Exercise**: How many rows does df contain?

In [ ]:
# ANSWER
df.count()

In [ ]:
df.rdd.getNumPartitions()

This corresponds to the number of cores in a free Google Colab instance.

In [ ]:
df.printSchema()

We use df.show() similarly to df.head() in Pandas.

In [ ]:
df.show()

In [ ]:
df.describe()

Once again evaluation of describe is lazy, we use show() to display results:

In [ ]:
df.describe().show()

Let us remove rows we do not need.

In [ ]:
df.select('VarID', 'Variant').distinct().show()

As we will not work with other variant, we can safely drop these columns.

In [ ]:
df = df.drop('VarID', 'Variant')

In [ ]:
df.show()

In [ ]:
df.select('Location').distinct().count()

In [ ]:
df.select('Location').distinct().show()

**Exercise**: Repeat the above query, this time ordering the results by Location and using the truncate=False option to display results in full. Show all 440 results.

In [ ]:
# ANSWER
df.select('Location').distinct().orderBy('Location').show(440, truncate=False)

Next run the following to confirm that there are no missing values in this dataset:

In [ ]:
df.count() - df.na.drop().count()

## Convert types

We saw above that all columns are in the form of strings. The following cell converts some of the columns to type float.

In [ ]:
floatcols = ['MidPeriod', 'PopMale', 'PopFemale', 'PopTotal']

for col_name in floatcols:
    df = df.withColumn(col_name, trim(col(col_name)).cast('float'))

**Exercise**: Similarly convert the four columns listed below into **integers**.

In [ ]:
intcols = ['Time', 'AgeGrp', 'AgeGrpStart', 'AgeGrpSpan']

#ANSWER
for col_name in intcols:
    df = df.withColumn(col_name, trim(col(col_name)).cast('int'))

In [ ]:
df.printSchema()

## Answering some queries about the data

We use filter to select a subset of rows satifying a True/False condition:

Example: What was the population breakdown by age and gender in Australia in 1970?

In [ ]:
df.filter((df.Location == 'Australia') & (df.Time == 1970)).show(101)

The select function can select a subset of columns.

**Exercise**: What was the population of 45-year-old females in India in 1960?

In [ ]:
# ANSWER
df.filter((df.AgeGrp == 45)\
          & (df.Location == 'India')\
          & (df.Time == 1960))\
          .select('PopFemale')\
          .show()

**Exercise**: Write a filter query to show the distinct locations starting with 'UN'.

In [ ]:
# ANSWER

df.filter(df.Location.startswith('UN'))\
  .select('Location')\
  .distinct()\
  .show(40, truncate=False)

Another example: what is the population of each location in 1950 and 2019?

In [ ]:
populations_in_1950 = df.filter(df.Time == 1950)\
                    .groupBy('Location')\
                    .sum('PopTotal')\
                    .withColumnRenamed('sum(PopTotal)', 'Population_1950')\
                    .withColumn('Population_1950', round('Population_1950', 3))\
                    .orderBy(col('Population_1950').desc())
populations_in_1950.show()

In [ ]:
populations_in_2019 = df.filter(df.Time == 2019)\
                    .groupBy('Location')\
                    .sum('PopTotal')\
                    .withColumnRenamed('sum(PopTotal)', 'Population_2019')\
                    .withColumn('Population_2019', round('Population_2019', 3))\
                    .orderBy(col('Population_2019').desc())


In [ ]:
populations_in_1950.join(populations_in_2019, 'Location').orderBy('Location').show()

**Exercise**: Which locations had the largest percentage change in population from 1950 to 2019?

In [ ]:
# ANSWER
populations_in_1950\
  .join(populations_in_2019, 'Location')\
  .orderBy('Location')\
  .withColumn('Percentage_change', 
              round(100*populations_in_2019.Population_2019\
              /populations_in_1950.Population_1950, 2))\
  .orderBy(col('Percentage_change').desc())\
  .show(truncate=False)

**Bonus Exercise**: In 2019 which locations have the highest percentage of seniors (age 80+) relative to their total population?

(Hint: if you find Japan, Greece and Italy you are right!)

In [ ]:
# ANSWER
seniors_2019 = df.filter((df.Time == 2019) & (df.AgeGrp >= 80))\
                  .groupBy('Location')\
                  .sum('PopTotal')\
                  .withColumnRenamed('sum(PopTotal)', 'Senior_Population_2019')\
                  .withColumn('Senior_Population_2019',\
                              round('Senior_Population_2019', 3))\
                  .orderBy(col('Senior_Population_2019').desc())

populations_in_2019\
  .join(seniors_2019, 'Location')\
  .withColumn('Percentage_of_Seniors', 
              round(100*seniors_2019.Senior_Population_2019\
              / populations_in_2019.Population_2019, 2))\
  .orderBy(col('Percentage_of_Seniors').desc())\
  .show(truncate=False)


## Population line plot

In this section, we use a query to perform a data visualisation with matplotlib. We shall plot population vs year for three countries.

In [ ]:
df_3countries = df[df.Location.isin("Australia", "New Zealand", "Singapore")]\
                  .select('Location', 'MidPeriod', 'PopTotal') \
                  .groupBy('Location', 'MidPeriod')\
                  .sum('PopTotal')\
                  .withColumnRenamed('sum(PopTotal)', 'Population')\
                  .withColumn('Population', round('Population', 3))
df_3countries.show()

In [ ]:
dataforplotting = df_3countries.toPandas()

Now that this dataset is of a manageable size, we convert it to a Pandas dataframe for plotting.

In [ ]:
dataforplotting.info()

In [ ]:
dataforplotting.pivot_table(index=['MidPeriod'], columns='Location', values='Population')

In [ ]:
ax = dataforplotting.pivot_table(index=['MidPeriod'], columns='Location', values='Population').plot()
ax.set_xlabel('Year');
ax.set_ylabel('Population in thousands');

**Bonus Exercise**: Plot the world population of children (age 0-17) along with those of the age groups 18-39 and 40+ from 1950 to 2019. You should display three lines (one for the population of each age group vs year) on the same plot. Note that 'World' is one of the locations.

Hint: one approach is to create a new column 'AgeCategory' based on AgeGrpStart using the 'when' function.

In [ ]:
# ANSWER
world_data = df[df.Location == "World"]\
                  .withColumn('AgeCategory',\
                              when(df.AgeGrpStart <= 17, '0-17')\
                              .when(df.AgeGrpStart >= 40, '40+')\
                              .otherwise('18-39'))\
                  .select('MidPeriod', 'AgeGrpStart',\
                          'AgeCategory', 'PopTotal') \
                  .groupBy('MidPeriod', 'AgeCategory')\
                  .sum('PopTotal')\
                  .withColumnRenamed('sum(PopTotal)', 'Population')\
                  .withColumn('Population', round('Population', 3))

In [ ]:
agegrp_df = world_data.toPandas()

In [ ]:
ax = agegrp_df.pivot_table(index=['MidPeriod'], columns='AgeCategory', values='Population').plot()
ax.set_xlabel('Year');
ax.set_ylabel('Population in thousands');

## Population Pyramid

In this section we show how a population pyramid may be created. We look at China in the year 1980.

In [ ]:
china_1980 = df.filter((df.Location == 'China (and dependencies)') & (df.Time == 1980)).toPandas()

In [ ]:
china_1980

In [ ]:
china_1980.describe()

Create age brackets in multiples of 5 - such as 0-4, 5-9, ...

In [ ]:
lower = china_1980['AgeGrpStart'] - (china_1980['AgeGrpStart'] % 5)

In [ ]:
agebrackets = [f'{x:02d}-{(x+4):02d}' for x in lower.values]
agebrackets[-1] = '100+'

In [ ]:
agebrackets[:10]

In [ ]:
china_1980['AgeRange'] = agebrackets

Next find the populations by age range.

In [ ]:
agg_china_1980 = (china_1980.groupby(['AgeRange']).sum()[['PopMale', 'PopFemale']]/1000).reset_index()

In [ ]:
rev_age = list(dict.fromkeys(agebrackets[::-1])) #reversed list of ages

In [ ]:
agg_china_1980['NegPopMale'] = -agg_china_1980['PopMale']

We are now ready to plot the population pyramid:

In [ ]:
sns.set(rc={'figure.figsize':(12,8)})
bar_plot = sns.barplot(x='NegPopMale', y='AgeRange', data=agg_china_1980, order = rev_age, color='red')
bar_plot = sns.barplot(x='PopFemale', y='AgeRange', data=agg_china_1980, order = rev_age, color='green')

labels = ["80", "60", "40", "20", "0", "20", "40", "60", "80"]
bar_plot.set_xticklabels(labels)

h = [bar_plot.bar(x=.1, height=.1, color = c) for c in ['red', 'green']] #used to set colour of bar in legend
bar_plot.legend(handles = h, labels=['Male', 'Female'], fontsize=20)

bar_plot.axes.set_title("Population Pyramid for China (1980)", fontsize=20);
bar_plot.set_xlabel("Population (millions)", fontsize=20);
bar_plot.set_ylabel("Age Group", fontsize=20);
bar_plot.set_xticklabels(bar_plot.get_xticklabels(), size = 15);
bar_plot.set_yticklabels(bar_plot.get_yticklabels(), size = 15);

## Prediction
Finally we use Spark's MLlib library in a linear regression problem. We shall predict the proportion of population to be of a particular age, given year.

Inputs:
- year
- age

Target variable:
- proportion of a country's population to be of that age, in that year


In [ ]:
pop_by_loc_and_year = df.groupBy('Location', 'Time')\
                        .sum('PopTotal')\
                        .withColumnRenamed('sum(PopTotal)', 'Location_year_pop')

In [ ]:
pop_by_loc_and_year.show()

In [ ]:
trainingset = df.join(pop_by_loc_and_year, ['Location', 'Time'])\
  .withColumn('Proportion',
              df.PopTotal/pop_by_loc_and_year.Location_year_pop)\
  .select('Time', 'AgeGrpStart', 'Proportion')

**Exercise**: Use the stat.corr function to find the correlation between AgeGrpStart and Proportion.

In [ ]:
# ANSWER
trainingset.stat.corr("AgeGrpStart", "Proportion")

Next we create a simple linear regression model predict Ratio from Time and AgeGrpStart.

In [ ]:
from pyspark.ml.regression import LinearRegression

MLlib takes input in vector form. Hence we need to create vectors from features.

In [ ]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=['Time', 'AgeGrpStart'],
                                 outputCol='Features')

In [ ]:
final_data_for_regression = featureassembler.transform(trainingset).select('Features', 'Proportion')
final_data_for_regression.show()

In [ ]:
train, test = final_data_for_regression.randomSplit([0.9,0.1])

In [ ]:
train

Next we cache these dataframes into memory:

In [ ]:
train = train.cache()

In [ ]:
test = test.cache()

We are ready to train the model. This may take a minute or so due to the size of training set.

In [ ]:
regressor = LinearRegression(featuresCol='Features', labelCol='Proportion')
regressor = regressor.fit(train)

In [ ]:
regressor.coefficients

In [ ]:
regressor.intercept

In [ ]:
predicted_results = regressor.evaluate(test)

Here we can compare some predicted proportions with actual proportions:

In [ ]:
predicted_results.predictions.show()

**Exercise**: Find R-squared and the mean squared error using the 'r2' and 'meanSquaredError' attributes.

In [ ]:
# ANSWER
predicted_results.r2, predicted_results.meanSquaredError

## Conclusion

We have seen how to use Pyspark to make queries on a large dataset, save smaller datasets into Pandas dataframe for plotting and how to use MLlib for machine learning on a large dataset.



---



---



> > > > > > > > > © 2023 Institute of Data


---



---



